<a href="https://colab.research.google.com/github/MiguelEuripedes/embedded_AI/blob/main/Projects/first_image_classifier/mlp_classifier/preprocessing_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your First Image Classifier: Using MLP to Classify Images

## Pre-processing

The purpose of this dataset is to correctly classify an image as containing a dog, cat, or panda. Containing only 3,000 images, the Animals dataset is meant to be another introductory dataset that we can quickly train a Multilayer Perceptron (MLP) model and obtain results that can be compared with the previously trained KNN model and the future CNN model.

Let's take the following steps:

1. Fetch Data (reuse of the previous project)
2. Pre-processing
3. Clean data

### Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:

* **pip install wandb** – Install the W&B library
* **import wandb** – Import the wandb library
* **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 29.1 MB/s 
     |████████████████████████████████| 182 kB 56.3 MB/s 
     |████████████████████████████████| 162 kB 55.6 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 162 kB 60.0 MB/s 
     |████████████████████████████████| 158 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 61.0 MB/s 
     |████████████████████████████████| 157 kB 54.0 MB/s 
     |████████████████████████████████| 157 kB 74.1 MB/s 
     |████████████████████████████████| 157 kB 57.7 MB/s 
     |████████████████████████████████| 157 kB 61.3 MB/s 
     |████████████████████████████████| 157 kB 61.0 MB/s 
     |████████████████████████████████| 157 kB 57.7 MB/s 
     |████████████████████████████████| 156 kB 50.6 MB/s 


In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

**Import the packages**

In [ ]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
import tensorflow as tf
import wandb

Configure logging reference for a logging object

In [ ]:
logger = logging.getLogger()

Set level of logging:

In [ ]:
logger.setLevel(logging.INFO)

Create handlers:

In [ ]:
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

Add handler to the logger:

In [ ]:
logger.handlers[0] = c_handler

### Step 02: Fetch Data

We'll be using the same strategy as before and replace our argument
parsing code with *hard coded* arguments and values

In [ ]:
args = {
    "dataset": "animals",
    "project_name": "first_image_classifier",
    "artifact_name": "animals_raw_data:latest",
}

Since we used the Fetch data step before, lets use it here. 
Lets open the W&B project created in the Fetch step

In [ ]:
run = wandb.init(entity="euripedes",project=args["project_name"], job_type="preprocessing")

wandb: Currently logged in as: euripedes. Use `wandb login --relogin` to force relogin


Download the raw data from W&B

In [ ]:
raw_data = run.use_artifact(args["artifact_name"])
data_dir = raw_data.download()
logger.info("Path: {}".format(data_dir))

wandb: Downloading large artifact animals_raw_data:latest, 187.97MB. 3000 files... 
wandb:   3000 of 3000 files downloaded.  
Done. 0:0:24.3
16-10-2022 02:37:37 Path: ./artifacts/animals_raw_data:v0


In [ ]:
run.finish()

### Step 03: Clean Data

Now lets create the proper MLP classifier. We start the project config with:

In [ ]:
data_dir

'./artifacts/animals_raw_data:v0'

Replacing our arguments parsing code with *hard coded* arguments and values

In [ ]:
args = {
    "features": "clean_features",
    "target": "labels",
    "project_name": "mlp_classifier",
}

Connecting to WandB with the new MLP project

In [ ]:
run = wandb.init(entity="euripedes",project=args["project_name"], job_type="preprocessing")

**Loader and Preprocessing Classes**

Source code based on Rosebrock, Adrian. Deep Learning For Computer vision with Python, 2019 [link](https://pyimagesearch.com/deep-learning-computer-vision-python-book/)

In [ ]:
# A basic simple preprocessor that resize a image

class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),interpolation=self.inter)

In [ ]:
# Building an image loader
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None, logger=None):
		# store the image preprocessor
    self.preprocessors = preprocessors
    self.logger = logger

		# if the preprocessors are None, initialize them as an
		# empty list
    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
    data = []
    labels = []

		# loop over the input images
    for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
			# e.g "img example: ./artifacts/animals_raw_data:v0/dogs/dogs_00892.jpg"
			# imagePath.split(os.path.sep)[-2] will return "dogs"
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      # check to see if our preprocessors are not None
      if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
        for p in self.preprocessors:
          image = p.preprocess(image)

			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
      data.append(image)
      labels.append(label)
   
			# show an update every `verbose` images
      if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        logger.info("[INFO] processed {}/{}".format(i + 1,len(imagePaths)))

		# return a tuple of the data and labels
    return (np.array(data), np.array(labels))

**Cleaning the data**

Grab the list of images that we'll be describing

In [ ]:
logger.info("[INFO] preprocessing images...")
imagePaths = list(paths.list_images(data_dir))

16-10-2022 02:40:18 [INFO] preprocessing images...


Initialize the image preprocessor, load the dataset from disk, and reshape the data matrix

In [ ]:
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp],logger=logger)
(data, labels) = sdl.load(imagePaths, verbose=500)
# 32 x 32 x 3 = 3072
data = data.reshape((data.shape[0], 3072))

16-10-2022 02:40:21 [INFO] processed 500/3000
16-10-2022 02:40:23 [INFO] processed 1000/3000
16-10-2022 02:40:24 [INFO] processed 1500/3000
16-10-2022 02:40:25 [INFO] processed 2000/3000
16-10-2022 02:40:27 [INFO] processed 2500/3000
16-10-2022 02:40:29 [INFO] processed 3000/3000


Show some information on memory consumption of the images

In [ ]:
logger.info("[INFO] features matrix: {:.1f}MB".format(data.nbytes / (1024 * 1024)))

16-10-2022 02:40:33 [INFO] features matrix: 8.8MB


Check the data shape and labels shape

In [ ]:
logger.info("Data shape: {}".format(data.shape))
logger.info("Label shape: {}".format(labels.shape))

16-10-2022 02:40:36 Data shape: (3000, 3072)
16-10-2022 02:40:36 Label shape: (3000,)


### Dumping the clean data artifacts to disk and upload to W&B

Save the feature artifacts and save the target, both using joblib

In [ ]:
joblib.dump(data, args["features"])
joblib.dump(labels, args["target"])
logger.info("Dumping the clean data artifacts to disk")

16-10-2022 02:40:40 Dumping the clean data artifacts to disk


**Clean data artifact:**

In [ ]:
artifact = wandb.Artifact(args["features"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean features data"
                          )

logger.info("Logging clean data artifact")
artifact.add_file(args["features"])
run.log_artifact(artifact)

16-10-2022 02:40:43 Logging clean data artifact


**Clean label artifact:**

In [ ]:
artifact = wandb.Artifact(args["target"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean target"
                          )

logger.info("Logging clean target artifact")
artifact.add_file(args["target"])
run.log_artifact(artifact)

16-10-2022 02:40:46 Logging clean target artifact


End the part of the project

In [ ]:
run.finish()